![MOSEK ApS](https://mosek.com/files/mosek_logo_color.png )



Inner and outer Löwner-John Ellipsoids
================================================

Computes the Löwner-John inner and outer ellipsoidal approximations of a polytope. 


Inner Löwner-John Ellipsoids
-----------------------------------------

The inner ellipsoidal approximation to a polytope 


\begin{equation}
    S = \{ x \in \mathbb{R}^n \mid Ax \leq b \}
\end{equation}

maximizes the volume of the inscribed ellipsoid,


.. math::  \{ x \mid x = Cu + d,   \| u \|_2 \leq 1 \}.


The volume is proportional to :math:`\mbox{det}(C)^{1/n}`, so the problem can be solved as 


.. math:: 

   \begin{array}{ll}
    \mbox{maximize}   &      t\\
    \mbox{subject to} &      t \leq \mbox{det}(C)^{1/n}\\
                      & \left\|C a_i\right\|_2 \leq b_i - a_i^Td,~ i=1,\ldots,m\\
                      & C \succeq 0
   \end{array}

which is equivalent to a mixed conic quadratic and semidefinite programming problem.


In [ ]:
def lownerjohn_inner(A, b):

    with Model("lownerjohn_inner") as M:
    
        m, n = len(A), len(A[0])

        # Setup variables
        t = M.variable("t", 1, Domain.greaterThan(0.0))
        C = M.variable("C", [n,n], Domain.unbounded())
        d = M.variable("d", n, Domain.unbounded())

        # (bi - ai^T*d, C*ai) \in Q, i=1..m
        M.constraint("qc", Expr.hstack(Expr.sub(b, Expr.mul(A,d)), Expr.mul(A,C.transpose())), Domain.inQCone())
        # t <= det(C)^{1/n}
        det_rootn(M, C, t)

        # Objective: Maximize t
        M.objective(ObjectiveSense.Maximize, t)

        M.solve()

        return ([C.level()[i:i+n] for i in range(0,n*n,n)], d.level())


Outer Löwner-John Ellipsoids
-----------------------------------------

The outer ellipsoidal approximation to a polytope given as the convex hull of a set of points


.. math::  S = \mbox{conv}\{ x_1, x_2, \ldots , x_m \}

minimizes the volume of the enclosing ellipsoid,  


.. math::  \{ x \mid \| P(x-c) \|_2 \leq 1 \}


The volume is proportional to :math:`\mbox{det}(P)^{-1/n}`, so the problem can be solved as


.. math:: 

   \begin{array}{ll}
   \mbox{minimize}   &    t\\
   \mbox{subject to} &  t  \geq \mbox{det}(P)^{-1/n}\\
                     & \left\|Px_i + c\right\|_2 \leq 1,~  i=1,\ldots,m\\
                     & P \succeq 0.
   \end{array}


References :cite:`ben-tal:01:a` for further reading.


In [25]:
from mosek.fusion import *
import math


def geometric_mean(M,x,t):

    def rec(x):
        n = x.shape.dim(0)
        if n > 1:
          y = M.variable(int(n//2), Domain.unbounded())
          M.constraint(Variable.hstack(Variable.reshape(x, NDSet(n//2,2)), y), Domain.inRotatedQCone())
          return rec(y)
        else:
          return x

  
    n = x.shape.dim(0)
    l = int(math.ceil(math.log(n, 2)))
    m = int(2**l) - n

  # if size of x is not a power of 2 we pad it:
    if m > 0:
        x_padding = M.variable(m,Domain.unbounded())

        # set the last m elements equal to t
        M.constraint(Expr.sub(x_padding, Variable.repeat(t,m)), Domain.equalsTo(0.0))

        x = Variable.vstack(x,x_padding)

    M.constraint(Expr.sub(Expr.mul(2.0**(l/2.0), t),rec(x)), Domain.equalsTo(0.0))

def det_rootn(M, X, t):
    n = int(math.sqrt(X.size()))

    # Setup variables
    Y = M.variable(Domain.inPSDCone(2*n))    
    
    # Setup Y = [X, Z; Z^T , diag(Z)] 
    Y11 = Y.slice([0, 0], [n, n])
    Y21 = Y.slice([n, 0], [2*n, n])
    Y22 = Y.slice([n, n], [2*n, 2*n])

    M.constraint( Expr.sub(Expr.mulElm( Matrix.diag(n,1.0) ,Y21), Y22), Domain.equalsTo(0.0) )
    M.constraint( Expr.sub(X, Y11), Domain.equalsTo(0.0) )

    # t^n <= (Z11*Z22*...*Znn)
    geometric_mean(M, Y22.diag(), t)


def lownerjohn_outer(x):
    with Model("lownerjohn_outer") as M:
        m, n = len(x), len(x[0])

        # Setup variables
        t = M.variable("t", 1, Domain.greaterThan(0.0))
        P = M.variable("P", NDSet(n,n), Domain.unbounded())
        c = M.variable("c", n, Domain.unbounded())

        # (1, P(*xi+c)) \in Q
        M.constraint("qc",
                     Expr.hstack(Expr.ones(m),
                                 Expr.sub(Expr.mul(x,P.transpose()),
                                          Variable.reshape(Variable.repeat(c,m), NDSet(m,2))
                                        )
                               ),
                     Domain.inQCone())

        # t <= det(P)^{1/n}
        det_rootn(M, P, t)

        # Objective: Maximize t
        M.objective(ObjectiveSense.Maximize, t)
        M.solve()

        return ([P.level()[i:i+n] for i in range(0,n*n,n)], c.level())

It is time now to generate some points:

In [ ]:
import random 

m = 100
n = 2

points = [ [ random.uniform(-1.,1.0) for j in range(n)] for i in range(m)]

P,c = lownerjohn_outer(points)